In [11]:
# - command: web IP; open website at IP

In [1]:
import enum
import ipaddress
import pathlib
import re

import pydantic

import utils.computer

In [2]:
NET_PATH = "test.json"

In [3]:
class DeviceType(enum.StrEnum):
    TERMINAL = "terminal"
    CYBERDECK = "cyberdeck"
    SERVER = "server"
    IMPLANT = "implant"

class Manufacturer(enum.StrEnum):
    OUTEL = "Outel"
    BMD = "BMD"
    CYCLOPS = "Cyclops"
    ACRON = "Acron"

class OperatingSystem(enum.StrEnum):
    ORACLEOS = "oracleOS"
    CYCLOPSOS = "CyclopsOS"

In [4]:
class NPv5Address:
    """Net Protocol v5 address. It is represented as 5 bytes seperated by dots."""

    def __init__(self, address: "str | int | NPv5Address") -> None:
        """Initialize the address."""
        self.address: int = self._parse(address)

    def __repr__(self) -> str:
        """Official string representation."""
        return f"{self.__class__.__name__}({self.address})"

    def __str__(self) -> str:
        """Informal string representation."""
        a, b, c, d, e = self.address.to_bytes(5)
        return f"{a:02X}.{b:02X}.{c:02X}.{d:02X}.{e:02X}"

    def _parse(self, address: "str | int | NPv5Address") -> int:
        """Parse address to int from str, int, or address."""
        if isinstance(address, NPv5Address):
            return address.address
        elif isinstance(address, str):
            if match := re.fullmatch(r"[0-9a-fA-F]{1,2}\.[0-9a-fA-F]{1,2}\.[0-9a-fA-F]{1,2}\."
                                     r"[0-9a-fA-F]{1,2}\.[0-9a-fA-F]{1,2}", address):
                return int.from_bytes(bytes.fromhex(match.string.replace(".", "")))
        elif isinstance(address, int):
            if 0 <= address <= 1099511627775:
                return address
        raise TypeError("Type not accepted.")

In [5]:
# A4.12.37.B0.F9
str(NPv5Address("A4.12.37.B0.F9"))

'A4.12.37.B0.F9'

In [6]:
SYSINFO_TEXT = """
OS:         {os}
Kernel:     {kernel}
Uptime:     {uptime}
Shell:      {shell}
CPU:        {cpu}
Memory:     {memory}
"""


class Device(pydantic.BaseModel):
    """Device respresentation."""
    # can be terminal, computer, server, brain computer, car, ...
    # do we use an actual type? (for game backend systems only)
    name: str
    """Set by user of the device."""
    type: DeviceType
    """Type of the device."""
    # hardware info
    manufacturer: Manufacturer
    """Name of the manufacturer."""
    device_name: str
    """Name of the device."""

    def sysinfo(self) -> str:
        return SYSINFO_TEXT.format_map({"os": "oracleOS", "kernel": "lunix", "uptime": "0d 3h 17m 9s",
                                        "shell": "bosh", "cpu": "Cyclops i803", "memory": "???"})

In [7]:
device = Device(name="zer0's Computer", type=DeviceType.TERMINAL, manufacturer=Manufacturer.ACRON,
                device_name="?")

In [8]:
print(device.sysinfo())


OS:         oracleOS
Kernel:     lunix
Uptime:     0d 3h 17m 9s
Shell:      bosh
CPU:        Cyclops i803
Memory:     ???



In [9]:
class Computer(pydantic.BaseModel):
    """Computer representation."""
    ip_address: ipaddress.IPv4Address = ipaddress.IPv4Address(1)
    ports: list[utils.computer.Ports] = []
    running_programs: list[None] = []  # actually program class? -> websites?
    # file_system: utils.computer.FileSystem = utils.computer.FileSystem()

In [10]:
class Net(pydantic.BaseModel):
    """Net/world representation."""
    home: Computer
    dns: dict[str, str]
    computers: list[Computer]
    network_graph: list[tuple[str, str]]

    def store(self) -> None:
        """Store the config in a file."""
        pathlib.Path(NET_PATH).write_text(self.model_dump_json(),
                                             encoding="utf-8")